In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import spacy

In [2]:
#importing the training data
df=pd.read_csv('IMDB Dataset.csv')
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [3]:
"""
sentiment : 0 = negative, 1 = positive 
use the following to get the sentiment of a sentence :  
sentiment = 0 if sentiment is negative else 1


use np.where to get the sentiment of a sentence :
"""
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [5]:
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']

In [6]:
"""
Load the spacy model and load the English language model from https://spacy.io/usage/models
"""
spacy.load("en_core_web_sm")

In [7]:
# general Settings

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.0001 ### ADD YOUR LEARNING RATE HERE ###
BATCH_SIZE = 100 ### ADD YOUR BATCH SIZE HERE ###
NUM_EPOCHS = 30 ### ADD YOUR NUMBER OF EPOCHS HERE ###
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 300 ### ADD YOUR EMBEDDING DIMENSION HERE ###
HIDDEN_DIM = 256 ### ADD YOUR HIDDEN DIMENSION HERE ###
NUM_CLASSES = 2

In [8]:
pip install torchtext==0.9

In [9]:
TEXT = torchtext.legacy.data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')

In [10]:
LABEL = torchtext.legacy.data.LabelField(dtype = torch.float)

In [11]:
df.to_csv('moviedata.csv', index = None)
df = pd.read_csv('moviedata.csv')
df.head()

,TEXT_COLUMN_NAME,LABEL_COLUMN_NAME
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [12]:
fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
                    path = '/content/IMDB Dataset.csv', ### ADD YOUR DATASET PATH HERE ###
                    format = 'csv', ### ADD YOUR DATASET FORMAT HERE ###
                    skip_header = False, ### ADD YOUR SKIP HEADER HERE ### 
                    fields =  fields### ADD YOUR FIELDS HERE ### 
)

In [13]:
train_data, test_data = dataset.split(split_ratio = [0.8, 0.2], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of test data', len(test_data))

Length of train data 40001
Length of test data 10000


In [14]:
train_data, val_data = train_data.split(split_ratio = [0.85, 0.15], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of valid data', len(val_data))

Length of train data 34001
Length of valid data 6000


In [15]:
print(vars(train_data.examples[2000]))

{'TEXT_COLUMN_NAME': ['Adorable', '!', 'I', 'saw', 'Domestic', 'Import', 'in', 'Philly', 'in', 'October', 'with', 'my', 'kids', '.', 'We', 'all', 'liked', 'it', 'so', 'much', 'that', 'we', 'saw', 'it', 'a', 'second', 'time', 'with', 'my', 'parents', '.', 'I', 'have', "n't", 'heard', 'them', 'laugh', 'like', 'that', 'in', 'years', '!', 'It', 'was', 'the', 'first', 'time', 'that', 'I', 'can', 'remember', 'seeing', 'a', 'movie', 'that', 'my', 'parents', 'and', 'my', 'kids', 'could', 'enjoy', '.', 'It', "'s", 'really', 'cute', 'and', 'we', 'ca', "n't", 'wait', 'for', 'it', 'to', 'come', 'out', 'on', 'DVD', '.', 'They', 'need', 'to', 'make', 'more', 'movies', 'like', 'Domestic', 'Import', '.', 'It', 'is', 'refreshing', 'to', 'go', 'to', 'a', 'movie', 'that', 'three', 'different', 'generations', 'can', 'enjoy', '(', 'and', 'not', 'be', 'embarrassed', ')', '.', 'I', 'have', 'not', 'seen', 'a', 'movie', 'this', 'cute', 'since', 'My', 'Big', 'Fat', 'Greek', 'Wedding', '.', 'I', 'loved', 'Mindy'

In [16]:
TEXT.build_vocab(train_data, max_size = VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'vocabulary size: {len(TEXT.vocab)}')
print(f'Label Size: {len(LABEL.vocab)}')

vocabulary size: 20002
Label Size: 3


In [17]:
most_common_words = TEXT.vocab.freqs.most_common(40)
print(most_common_words)

[('the', 392110), (',', 371221), ('.', 320075), ('a', 210542), ('and', 210314), ('of', 195115), ('to', 180616), ('is', 146429), ('in', 118677), ('I', 106550), ('it', 103775), ('that', 94301), ('"', 86391), ("'s", 82965), ('this', 82095), ('-', 70222), ('/><br', 69261), ('was', 67888), ('as', 58025), ('movie', 57912), ('with', 57762), ('for', 56443), ('film', 52340), ('The', 51295), ('but', 46913), ("n't", 44701), ('on', 44578), ('(', 44459), (')', 44091), ('you', 42067), ('are', 40610), ('not', 38989), ('have', 38503), ('his', 37675), ('be', 36184), ('one', 32993), ('he', 32886), ('!', 30128), ('at', 29700), ('by', 29439)]


In [18]:
print(TEXT.vocab.itos[:20])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is', 'in', 'I', 'it', 'that', '"', "'s", 'this', '-', '/><br', 'was']


In [19]:
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data), ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
        batch_size = BATCH_SIZE, ### ADD YOUR BATCH SIZE HERE ###
        sort_within_batch = True, ### ADD YOUR SORT WITHIN BATCH HERE ### 
        sort_key = lambda x : len(x.TEXT_COLUMN_NAME), 
        shuffle = True
    )

In [20]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([255, 100])
Target vector size: torch.Size([100])

Valid:
Text matrix size: torch.Size([54, 100])
Target vector size: torch.Size([100])

Test:
Text matrix size: torch.Size([49, 100])
Target vector size: torch.Size([100])


In [21]:
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data ), 
        ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
        batch_size = BATCH_SIZE, 
        ### ADD YOUR BATCH SIZE HERE ###
        sort_within_batch = True, 
        ### ADD YOUR SORT WITHIN BATCH HERE ###
        sort_key = lambda x : len(x.TEXT_COLUMN_NAME),
        device = DEVICE,
        shuffle = True
    )

In [58]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)  
        self.rnn = torch.nn.LSTM(embedding_dim, hidden_dim)
        self.drop = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        self.sig = torch.nn.Sigmoid()
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        out = self.drop(torch.tensor(hidden[0]))
        output = self.fc(out)
        output = self.sig(output)
        return output

In [59]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim= len(TEXT.vocab), 
            embedding_dim= EMBEDDING_DIM, 
            hidden_dim= HIDDEN_DIM, 
            output_dim= 1 
)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.BCEWithLogitsLoss()

In [24]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [60]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)
        optimizer.zero_grad()
        predictions = model(text).view(BATCH_SIZE, -1)
        predictions = predictions.squeeze(1)
        loss = criterion(predictions, labels) 
        loss.backward()
        optimizer.step()
        
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Epoch: 001/030 | Batch 000/341 | Loss: 0.7153
Epoch: 001/030 | Batch 050/341 | Loss: 0.6956
Epoch: 001/030 | Batch 100/341 | Loss: 0.6944
Epoch: 001/030 | Batch 150/341 | Loss: 0.6910
Epoch: 001/030 | Batch 200/341 | Loss: 0.6908
Epoch: 001/030 | Batch 250/341 | Loss: 0.7087
Epoch: 001/030 | Batch 300/341 | Loss: 0.6977


RuntimeError: ignored

In [ ]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

In [ ]:
print('Probability positive:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")